**Section 2: Data Preparation**

Notebook for "Introduction to Data Science and Machine Learning"

version 1.0, April 30 2024

**Information:**

In this task we will work with a data set that offers possibilities of data cleaning. The data set consists of some dummy data that was produced specifically for this lab assignment. So it is not meaningful. 

## Importing Basic Modules

We will need some basic modules:

- `pandas` implements the data frames we will look at
- `seaborn` provides statitical plots and some basic data sets like the iris flower data set, and
- `matplotib.pyplot` provides basic (MATLAB like) plotting functionality.

We will thus import all four modules first.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import scatter_matrix
from pandas.api.types import CategoricalDtype

# Data Preparation

## 1. Loading the data

We start by loading the dataset. It is defined in a `csv` file so we load it directly to a pandas data frame using `read_csv()`.


In [ ]:
data=pd.read_csv('data/dataPreparation.csv')

In [ ]:
data

## 2. Getting to know the data

Please use the code elements you learned in the notebook "1 Getting to Know the Data":

- Display the first five rows of the data

- Display the information about the data 

**Questions:**
- how many data entries are there?
- how many columns are there?
- are there any missing values? How many? In which column?

- Display the statistical information of the numerical data

Execute the following code. Does this plot display useful information?

In [ ]:
data.plot.box()

Create a scatter matrix for the data:

**Question:** Are there any attributes that might be correlated?

## 3. Filling in missing values

In the data frame we have missing values: each one value is missing in column `B`, `D` and `E`.

We will now look at three different methods to deal with the missing values. As the data set is random, this is used to demonstrate how we could handle missing values. For real data we need of course to carefully investigate the data and make a well informed decision.

As a hint we will always mention "*after careful deliberation we decided to*".

First we are interested in taking a look at the rows with `na`s. In order to do so we use Boolean indexing. We write a function so we can call it later to easily check whether we solved all problems:

In [ ]:
def printNARows(data):
    # First we create the Boolean mask for na values
    mask=pd.isna(data)
    # We remember that Boolean values are numeric data types: True has the value of 1 and False of 0
    # we now create a vector with the sums of the rows
    sums=mask.sum(axis=1)
    # and now we select the rows where the sum is >=1 (i.e. there is at least one na)
    naFrame=data[sums>0]
    print(naFrame)

In [ ]:
# We call the function
printNARows(data)

We now assume we carefully investigated the rows with the missing data.
Lets take a look at the different values of column `B`:

In [ ]:
data['B'].unique()

The values are either `yes` or `no`. We count the values:

In [ ]:
data['B'].value_counts()

The method `value_counts()` ignores `na`s. *After careful deliberation we decided to* **replace** the `na` with **the most frequent value**, that is a `no`.

In [ ]:
data.loc[15,'B']='no'

And now we again display the `na` rows:

In [ ]:
printNARows(data)

*After careful deliberation we decided to* **replace** the `na` for the `D` value in row 8 **with the mean value** of the column:

In [ ]:
data.loc[8,'D']=data['D'].mean()

And we check again the `na` rows:

In [ ]:
printNARows(data)

Unfortunately we cannot solve the problem with the last `na`. We assume that this is an indication for an errorneous data collection so *after careful deliberation we decided to* **drop the whole row**:

In [ ]:
dataNoNA=data.dropna()
printNARows(dataNoNA)

We will now continue working with `dataNoNA`.

### 3.1 `pandas` Methods

In our example we looked at `na` values individually and decided what to do. The `pandas` module offers several methods to handle `na`-values automatically:
- bfill(): Fill NA/NaN values by using the next valid observation to fill the gap.  (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html)
- ffill(): Fill NA/NaN values by propagating the last valid observation to next valid. (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html)
- dropna(): Remove missing values. (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)
- fillna(): Fill NA/NaN values using the specified method.(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)
- interpolate(): Fill NaN values using an interpolation method. (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html)

## 4. Examining Column `A`

Let's take a look at the boxplot of column `A` alone:

In [ ]:
dataNoNA['A'].plot.box()

It looks very regular. But this should be no surprise: when we take a closer look at the data, we realize that it is an index. So this column does not contain any specific information. We evaluate the situation and determine that it is not important to keep this index for our analysis. Therefore, we simply **drop the column** (of course if we needed to refer back to the data object, we should keep the index. In this assignment we simply do many things to learn how they are done!) and save the result in a new data frame `df1`. 

In [ ]:
df1=dataNoNA.drop('A',axis=1)

In [ ]:
df1.head()


## 5. Examining Column `B`

Let's take a look at column `B`. 

In [ ]:
df1.B.unique()

This column has two different values. As `dtype` `object` is specified. But by understanding the data we realize that the data is categorical. So we add a new column with categorical data.

**Hint**: In fact this categorical data is nominal, as we cannot specify an order.

In [ ]:
df1['B_cat']=df1['B'].astype('category')

In [ ]:
df1.info()

**Hint:** There is no general rule whether you should keep the original column and add a new one or, after adding the column with categorical B values delete the original B column. In this example we will delete (drop) the original column.

In [ ]:
df1=df1.drop('B',axis=1)

## 6. Examining Column `C`

Let's take a look at the boxplot for column `C`.

In [ ]:
df1.C.plot.box()

It really looks a bit weird. When we look at the numerical data (type `int64`) we see that there are only two different values:

In [ ]:
df1.C.value_counts()

When investigating the attribute we learn that the values `1` and `0` are not numerical but *ordinal* attributes, with `0` < `1`. We want that the values are treated as categories and not as numbers, as average values, etc., are not defined (i.e. do not make sense). Therefore, we create a new column for the `C` attribute as ordinal type:

In [ ]:
cat_dtype = CategoricalDtype(categories=[0,1], ordered=True)

df1['C_ord']=df1.C.astype(cat_dtype)

In [ ]:
df1.info()

Please check out the differences.

In [ ]:
df1.B_cat.dtype

In [ ]:
df1.C_ord.dtype

In [ ]:
df1.describe()

**Hint:** We will equally drop the original C column, as we do not one any average values for the ordinal values 0 and 1 to be displayed in a summary like the one above.

In [ ]:
df1=df1.drop('C',axis=1)

## 7. Examining Columns `D` and `E` 

The columns `D` and `E` are numerical floating point data. We can take a look at the boxplots. In order to only display the boxplots for those two data, we need to create a new data frame

In [ ]:
(df1.loc[:,['D','E']]).plot.box()

## 8. Examining the `age` column

Column `age` contains the age in form of an integer. Let's take a look at the boxplot:


In [ ]:
(df1.loc[:,['age']]).plot.box()

This looks weird. So lets check the statistical summary of data:


In [ ]:
df1.age.describe()

We have a negative `age` value which is apparently wrong. 

Now let's take a look at the row(s) with a negative age:

In [ ]:
df1.loc[df1.age<0,:]

We see that it is one single row. There must be a data entry error. We do not know, how to fix it, so we drow the row:


In [ ]:
df1=df1.drop(16,axis=0)

and check the result:

In [ ]:
df1.age.describe()

A histogram might be helpful to display the age disctribution: 

In [ ]:
sns.histplot(df1.age,kde=True)

## 9. Examining the column `gender`

Now let's take a look at the column `gender`. First we will display the unique values:

In [ ]:
df1.gender.unique()

And here we see now a problem that might be a result of data entry: `F` and `female` are both values for  `female`. `M`, `m` and `male` are all values for `male`. Additionally the attribute is an `object` and not a category.

First we will fix the wrong entries by replacing the values with the correct values.

To do so we use the method `replace()`. Using the parameter `inplace=True` we modify the original data frame.

In [ ]:
df1.replace({"gender":{"F":"female","M":"male","m":"male"}}, inplace=True)

The `replace()` method expects a dictionary. A dictionary ist defined inside `{}` and consists of key-value pairs. First the key is specified followed by a `:` and the value.
In above code we state that we want to apply the `replace()` method to the column `gender`. As value we specify a second dictionary that contains as keys the values to be replaces and as values the new values. So we state that "F" should be replaced by "female", "M" by "male" and "m" by "male". The singe key-value pairs are separated by commas. 

We coulds equally specify values in other columns to be replaced.

With the keyword argument `inplace=True` we speciify that the values should be replaced in the original data frame and not on a copy, that would then be returned by the method.

And we control the result:

In [ ]:
df1.gender.unique()

Now we need to modify the type of the column to a category (not an ordinal value). Please proceed as explained above and create a new column named `genderCat` and drop the origial `gender` category:

In [ ]:
# your solution


Let's check the result:

In [ ]:
df1.info()

## 9. Examining the column `country`

Let's take a look at the values of the column `country`:

In [ ]:
df1.country.unique()

Here we equally have the problem of errornous entries. Please correct the wrong entries as above. Use `India`, `Germany`, `France` and `UK` as the correct entries: 

In [ ]:
# your solution


Let's check the solution:

In [ ]:
df1.country.unique()

Let's count the data objects per value:

In [ ]:
df1.country.value_counts()

And now let's create a plot that displays the counts of the country data

In [ ]:
sns.countplot(df1,x='country')

We can now distinguish the number of counts per gender (please ignore the Furure Warning):

In [ ]:
sns.countplot(df1,x='country',hue='genderCat')

Here we can, e.g. see, that the gender distribution differs significantly per country. (Keep in mind, this is just an artificial data set for this lab assignment, so this does not really mean anything.)

We can equally display the percentage:

In [ ]:
sns.countplot(df1,x='country',hue='genderCat',stat="percent")

Now we want to modify the type of the `country` column. The `country` shall be (for whatever reason) an ordinal attribute. The order is defined by the population size in descending order (India, Germany, UK, France). 

In [ ]:
# Your solution

## 10. Some statistical information

As already seen, we display statistical information for numeric / quantitative attributes using `describe()`:

In [ ]:
df1.describe()

Let's take a look at the mean values of `D` and `E`:

In [ ]:
df1[['D','E']].mean()

With `groupby()` we can calculate this information (or the sum or the minimum etc) per one attribute, e.g. the county:

In [ ]:
df1.groupby('country')[['D','E']].mean()

`groupby()` equally works on different levels:

In [ ]:
df1.groupby(['country','genderCat','B_cat'], observed=False)[["D","E"]].mean()

In two lines you can see an `NaN`. This indicates that there are no data object with the respective values of country, gender and `B`. This is not displayed if we set observed to `True`.

In [ ]:
df1.groupby(['country','genderCat','B_cat'],observed=True)[["D","E"]].mean()

Many Machine Learning algorithms cannot cope with multivalues categorical attributes. For this reason often one hot encoding is used to 

In [ ]:
df1New=pd.get_dummies(data=df1,columns=["country"])

In [ ]:
df1New

Now we can store the cleaned data frame to a **csv* file:

In [ ]:
df1New.to_csv("cleanedData.csv")

**Further Information:**

Often it is important to identify duplcates and remove them. `pandas` offers the following methods:
- `duplicated()`: Return boolean Series denoting duplicate rows. Considering certain columns is optional. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html
- `drop_duplicates()`: Return DataFrame with duplicate rows removed. Considering certain columns is optional. Indexes, including time indexes are ignored. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

It is highly recommended to first look at the duplicates before dropping them. Not all duplicates that are identified by `pandas` automatically are indeed duplicates that can be dropped.

## Exercise

Please load the file `dataPreparation_Exercise.csv`. It is an artificially created data set with information on course, study hours for the final oral examination, gender, country of origin, age and grades.

Check out `nan` values and discuss possible actions, clean the data, transform categorical values to categories and get to know the data set. There are duplicate data rows. Create some plots and summarize statistical information using `groupby()`.

Enjoy!

In [ ]:
# Your Code
data2=pd.read_csv('data/dataPreparation_Exercise.csv')


<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This notebook was created by Christina B. Class for teaching at EAH Jena and is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.